https://www.kaggle.com/datasets/laotse/credit-risk-dataset

In [195]:
# kütüphaneleri ekledim
import pandas as pd, joblib
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier



In [196]:
#datasetimi koduma ekledim
df = pd.read_csv('credit_risk_dataset.csv')

In [197]:
#datasetimden ilk 5 veriyi kontrol ettim
df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [198]:
#sütun isimlerini türkçeye çevirdim
df.rename(columns={
    'person_age': 'yas',
    'person_income': 'gelir',
    'person_home_ownership': 'evi',
    'person_emp_length': 'calisma_suresi',
    'loan_intent': 'kredi_amaci',
    'loan_grade': 'kredi_notu',
    'loan_amnt': 'kredi_miktarı',
    'loan_int_rate': 'kredi_faiz_orani',
    'loan_status': 'kredi_durumu',
    'loan_percent_income': 'kredi_yuzdesi_geliri',
    'cb_person_default_on_file': 'kredi_gecmisi_temerrut_var_mi',
    'cb_person_cred_hist_length': 'kredi_gecmis_suresi_yil'
}, inplace=True)

In [199]:
#sütün isimlerini kontrol ettim
df.dtypes

yas                                int64
gelir                              int64
evi                               object
calisma_suresi                   float64
kredi_amaci                       object
kredi_notu                        object
kredi_miktarı                      int64
kredi_faiz_orani                 float64
kredi_durumu                       int64
kredi_yuzdesi_geliri             float64
kredi_gecmisi_temerrut_var_mi     object
kredi_gecmis_suresi_yil            int64
dtype: object

In [200]:
#eksik veri sayısına baktım
df.isnull().sum()

yas                                 0
gelir                               0
evi                                 0
calisma_suresi                    895
kredi_amaci                         0
kredi_notu                          0
kredi_miktarı                       0
kredi_faiz_orani                 3116
kredi_durumu                        0
kredi_yuzdesi_geliri                0
kredi_gecmisi_temerrut_var_mi       0
kredi_gecmis_suresi_yil             0
dtype: int64

In [201]:
# calisma_suresi ve kredi_faiz_orani sütunlarını ortalama ile doldurdum
df['calisma_suresi'].fillna(df['calisma_suresi'].median(), inplace=True)
df['kredi_faiz_orani'].fillna(df['kredi_faiz_orani'].median(), inplace=True)

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_12228\1639192107.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['calisma_suresi'].fillna(df['calisma_suresi'].median(), inplace=True)
C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_12228\1639192107.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

In [202]:
#tekrardan kalan eksik veri varmı diye kontrol ettim
df.isnull().sum()

yas                              0
gelir                            0
evi                              0
calisma_suresi                   0
kredi_amaci                      0
kredi_notu                       0
kredi_miktarı                    0
kredi_faiz_orani                 0
kredi_durumu                     0
kredi_yuzdesi_geliri             0
kredi_gecmisi_temerrut_var_mi    0
kredi_gecmis_suresi_yil          0
dtype: int64

In [203]:
# kategorik verileri sütuna dönüştürdüm 
df = pd.get_dummies(df,
                    columns=['evi', 'kredi_amaci', 'kredi_notu', 'kredi_gecmisi_temerrut_var_mi'],
                    drop_first=True
)

In [204]:
# güncel sütun listemi kontrol ettim
df.columns.tolist()

['yas',
 'gelir',
 'calisma_suresi',
 'kredi_miktarı',
 'kredi_faiz_orani',
 'kredi_durumu',
 'kredi_yuzdesi_geliri',
 'kredi_gecmis_suresi_yil',
 'evi_OTHER',
 'evi_OWN',
 'evi_RENT',
 'kredi_amaci_EDUCATION',
 'kredi_amaci_HOMEIMPROVEMENT',
 'kredi_amaci_MEDICAL',
 'kredi_amaci_PERSONAL',
 'kredi_amaci_VENTURE',
 'kredi_notu_B',
 'kredi_notu_C',
 'kredi_notu_D',
 'kredi_notu_E',
 'kredi_notu_F',
 'kredi_notu_G',
 'kredi_gecmisi_temerrut_var_mi_Y']

In [205]:
#aykırı değerleri çıkartmadan önceki veri sayısına bakıyorum
df.shape

(32581, 23)

In [206]:
#aykırı değerleri kontrol ediyorum ve anlamsız verileri çıkartıyorum
df = df[df['yas'] <= 100]
df = df[df['calisma_suresi'] <= 60]

In [207]:
#aykırı değerleri çıkarttıktan sonraki veri sayısına bakıyorum
df.shape

(32574, 23)

In [208]:
df = df.astype(int)
df

,yas,gelir,calisma_suresi,kredi_miktarı,kredi_faiz_orani,kredi_durumu,kredi_yuzdesi_geliri,kredi_gecmis_suresi_yil,evi_OTHER,evi_OWN,...,kredi_amaci_MEDICAL,kredi_amaci_PERSONAL,kredi_amaci_VENTURE,kredi_notu_B,kredi_notu_C,kredi_notu_D,kredi_notu_E,kredi_notu_F,kredi_notu_G,kredi_gecmisi_temerrut_var_mi_Y
1,21,9600,5,1000,11,0,0,2,0,1,...,0,0,0,1,0,0,0,0,0,0
2,25,9600,1,5500,12,1,0,3,0,0,...,1,0,0,0,1,0,0,0,0,0
3,23,65500,4,35000,15,1,0,2,0,0,...,1,0,0,0,1,0,0,0,0,0
4,24,54400,8,35000,14,1,0,4,0,0,...,1,0,0,0,1,0,0,0,0,1
5,21,9900,2,2500,7,1,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,1,5800,13,0,0,30,0,0,...,0,1,0,0,1,0,0,0,0,0
32577,54,120000,4,17625,7,0,0,19,0,0,...,0,1,0,0,0,0,0,0,0,0
32578,65,76000,3,35000,10,1,0,28,0,0,...,0,0,0,1,0,0,0,0,0,0
32579,56,150000,5,15000,11,0,0,26,0,0,...,0,1,0,1,0,0,0,0,0,0


In [209]:
#güncel verileri yeni csv dosyasına kaydediyorum
df.to_csv('credit_risk_dataset_main.csv')

In [210]:
#güncel verileri yeni excel dosyasına kaydediyorum
df.to_excel('credit_risk_dataset_main.xlsx')

In [211]:
df.head()

,yas,gelir,calisma_suresi,kredi_miktarı,kredi_faiz_orani,kredi_durumu,kredi_yuzdesi_geliri,kredi_gecmis_suresi_yil,evi_OTHER,evi_OWN,...,kredi_amaci_MEDICAL,kredi_amaci_PERSONAL,kredi_amaci_VENTURE,kredi_notu_B,kredi_notu_C,kredi_notu_D,kredi_notu_E,kredi_notu_F,kredi_notu_G,kredi_gecmisi_temerrut_var_mi_Y
1,21,9600,5,1000,11,0,0,2,0,1,...,0,0,0,1,0,0,0,0,0,0
2,25,9600,1,5500,12,1,0,3,0,0,...,1,0,0,0,1,0,0,0,0,0
3,23,65500,4,35000,15,1,0,2,0,0,...,1,0,0,0,1,0,0,0,0,0
4,24,54400,8,35000,14,1,0,4,0,0,...,1,0,0,0,1,0,0,0,0,1
5,21,9900,2,2500,7,1,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0


## Modelleri Kaydetmek İçin Bir Definition Oluşturdum 

In [212]:
def saveModel(model, name: str):
    joblib.dump(model, 'models/'+name)
    return f"Model başarıyla {name} olarak kaydedildi."

def getModel(name: str):
    return joblib.load('models/'+name)


# Karar Ağaçları Model Eğitimi

In [213]:
X = df.drop('kredi_durumu', axis=1)
y = df['kredi_durumu']

In [214]:
# Verilerimin 20% sini test verisi olarak ayırdım
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [215]:
# Model eğitimini başlattım
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

saveModel(model, 'kararAgaclariModel.pt')


'Model başarıyla kararAgaclariModel.pt olarak kaydedildi.'

In [216]:
# Modeli Test ediyorum
y_pred = model.predict(X_test)

In [217]:
# Modelin Performansı
print(
f"Karar Ağaçları\n\
Doğruluk: {round(accuracy_score(y_test, y_pred), 2)}\n\
Kesinlik: {round(precision_score(y_test, y_pred), 2)}\n\
Duyarlılık: {round(recall_score(y_test, y_pred), 2)}")

Karar Ağaçları
Doğruluk: 0.89
Kesinlik: 0.74
Duyarlılık: 0.78


# Random Forest Model Eğitimi

In [218]:
# Modeli Eğittim
rf_model = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_model.fit(X_train, y_train)

saveModel(rf_model, 'randomForestModel.pt')


'Model başarıyla randomForestModel.pt olarak kaydedildi.'

In [219]:
# Modeli Test Ediyorum
y_pred_rf = rf_model.predict(X_test)

In [220]:
print(
f"Random Forest\n\
Doğruluk: {round(accuracy_score(y_test, y_pred_rf), 2)}\n\
Kesinlik: {round(precision_score(y_test, y_pred_rf), 2)}\n\
Duyarlılık: {round(recall_score(y_test, y_pred_rf), 2)}")

Random Forest
Doğruluk: 0.93
Kesinlik: 0.92
Duyarlılık: 0.73


# XGBoost Model Eğitimi

In [221]:
# Modeli Eğittim
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

saveModel(xgb_model, 'xgboostModel.pt')

'Model başarıyla xgboostModel.pt olarak kaydedildi.'

In [222]:
# Modeli Test Ediyorum
y_pred_xgb = xgb_model.predict(X_test)

In [223]:
print(
f"XGBoost\n\
Doğruluk: {round(accuracy_score(y_test, y_pred_xgb), 2)}\n\
Kesinlik: {round(precision_score(y_test, y_pred_xgb), 2)}\n\
Duyarlılık: {round(recall_score(y_test, y_pred_xgb), 2)}")

XGBoost
Doğruluk: 0.93
Kesinlik: 0.93
Duyarlılık: 0.74
